# Cálculo de derivadas temperatura del suelo -10cm y graficación

> Elaborado por Paola Álvarez, profesional contratista IDEAM, contrato 196 de 2024. Comentarios o inquietudes, remitir a *palvarez@ideam.gov.co* 

**Librerías**

In [1]:
import pandas as pd
import numpy as np
import datetime
import statistics
import glob
import os
import csv
import re
import gc
import calendar
from collections import deque
from datetime import timedelta
from scipy import stats
from openpyxl import Workbook
from openpyxl.chart import LineChart, Reference
from openpyxl.chart import ScatterChart, Reference, Series
from openpyxl.chart import BarChart, Reference
from openpyxl.utils.dataframe import dataframe_to_rows

____

### Pruebas unitarias

#### Datos con QC

In [9]:
# Función para procesar frecuencias
def process_frequencies(df, columna_fecha, freq_csv_path, porc_min=0.67):
    # Convertir la columna de fecha a datetime si aún no lo es
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
        
    # Cargar el archivo de frecuencias
    freqinst200b = pd.read_csv(freq_csv_path, encoding='latin-1')

    # Definir el diccionario de frecuencias y cantidades esperadas
    frecuencias = {
        'T': {'cant_esperd_h': 60, 'cant_esperd_d': 1440, 'cant_esperd_m': 43200, 'cant_esperd_a': 518400, 'minutos': 1},
        '5T': {'cant_esperd_h': 12, 'cant_esperd_d': 288, 'cant_esperd_m': 8640, 'cant_esperd_a': 103680, 'minutos': 5},
        '10T': {'cant_esperd_h': 6, 'cant_esperd_d': 144, 'cant_esperd_m': 4320, 'cant_esperd_a': 51840, 'minutos': 10},
        'h': {'cant_esperd_h': 1, 'cant_esperd_d': 24, 'cant_esperd_m': 720, 'cant_esperd_a': 8640}
    }

    # Obtener el valor de la estación
    station_value = df['Station'].values[0]
    freqinst200b_station = freqinst200b.loc[freqinst200b['Station'] == station_value]
    periodos = freqinst200b_station['FreqInf'].values[0]

    if pd.isna(periodos):
        try:
            periodos = pd.infer_freq(df[columna_fecha][-25:])
            print(periodos)
            if periodos is None:
                print(f"Frecuencia inferida es None para el archivo {df}")
                return None, None, None, None
        except ValueError as e:
            print(f'Error al inferir la frecuencia en el archivo {df}: {str(e)}')
            return None, None, None, None

    # Obtener las cantidades esperadas y el offset en minutos
    cant_esperd_h = frecuencias[periodos]['cant_esperd_h']
    cant_esperd_d = frecuencias[periodos]['cant_esperd_d']
    cant_esperd_m = frecuencias[periodos]['cant_esperd_m']
    cant_esperd_a = frecuencias[periodos]['cant_esperd_a']
    if periodos == 'h':
        pass
    else:
        minutoffset = frecuencias[periodos]['minutos']

    # Ajustar la hora de cada registro
    df_c = df.copy()
    if periodos == 'h':
        df_c[columna_fecha] = df_c[columna_fecha] - pd.Timedelta(hours=1)
    else:
        df_c[columna_fecha] = df_c[columna_fecha] - pd.Timedelta(minutes=minutoffset)

    # Establecer la columna de fecha como índice
    df_c.set_index(columna_fecha, inplace=True)

    # Función para verificar si un día, mes o año tiene suficientes datos
    def complet_dia(sub_df):
        return len(sub_df) >= cant_esperd_d * porc_min

    def complet_mes(sub_df):
        return len(sub_df) >= cant_esperd_m * porc_min

    def complet_anio(sub_df):
        return len(sub_df) >= cant_esperd_a * porc_min

    return df_c, complet_dia, complet_mes, complet_anio

# Uso funciones de frecuencias
df_example = pd.read_csv('../../OE_3_QC_Variables/2_HumedadRelativa/Test_QC/Estacion_0011025501_qc.csv', encoding='latin-1')#, dtype={'Estado_Anterior':str})
if 'Estado' in df_example.columns:
    df_example = df_example[~df_example['Estado'].apply(lambda x: any([str(x).startswith(prefix) for prefix in ['0PSO','0PAT','0PER']]))]
    
columna_fecha = 'Fecha'
freq_csv_path = '../../OE_3_QC_Variables/2_HumedadRelativa/EMAhrs10_LatLonEntFreq2.csv'

In [10]:
# V2_22/07/2024, cambio de procesamiento de frecuencias
## Derivadas diarias, mensuales y anuales
# Humedad relativa del aire a 10 cm media diaria
df_example_c = df_example.copy()
def HRS10_MEDIA_D(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
     
    # Se filtran los que hayan superaddo las pruebas
    #dfC = df[df['Estado'].apply(lambda x: any([str(x).startswith(prefix) for prefix in ['0PC']]))]
    # Se llama la función de procesamiento de frecuencias
    df_c, complet_dia, _, _ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)

    if df_c is None or complet_dia is None:
        return None

    # Se filtran los datos que tienen la complititud mínima
    df_filtrado = df_c.groupby([df_c.index.date]).filter(complet_dia) #dfC.groupby([dfC.index.date]).filter(complet_dia)
    hrs10_med_d = df_filtrado[[columna_valor]].resample('D').mean()

    return hrs10_med_d

# Ejemplo de uso de la función
dfhrs10_med_d = HRS10_MEDIA_D(df_example_c)

# Presión atmosférica media mensual
def HRS10_MEDIA_M(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):    
    df.reset_index(inplace=True)
    # Convertir la columna de fecha a datetime si aún no lo es
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
    
    # Establecer la columna 'Fecha' como índice
    df.set_index(columna_fecha, inplace=True)
    
    days_in_month = df.index.to_series().dt.days_in_month
    days_in_month = days_in_month.resample('ME').first()
    
    # Función para verificar si una hora específica tiene suficientes datos
    def complet_mes(sub_df):
        mes = sub_df.index[0].month
        total_esperado = days_in_month[days_in_month.index.month == mes].iloc[0]
        return len(sub_df) >= total_esperado * porc_min

    # Luego de establecer el índice, aplicar resample
    df_filtrado = df.groupby([df.index.year, df.index.month]).filter(complet_mes)
    hrs10_med_m = df_filtrado[['Valor']].resample('ME').mean()
    
    return hrs10_med_m

dfhrs10_med_m = HRS10_MEDIA_M(dfhrs10_med_d)

# Presión atmosférica del Aire a 2 metros media anual
def HRS10_MEDIA_A(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
    df.reset_index(inplace=True)
    # Convertir la columna de fecha a datetime si aún no lo es
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
    
    # Función para verificar si una hora específica tiene suficientes datos
    def complet_anio(sub_df):
        return len(sub_df) >= 12 * porc_min
       
    # Antes de resample, establecer la columna 'Fecha' como índice
    df.set_index(columna_fecha, inplace=True)

    # Luego de establecer el índice, aplicar resample
    df_filtrado = df.groupby([df.index.year]).filter(complet_anio)
    hrs10_med_a = df_filtrado[['Valor']].resample('YE').mean()
    
    return hrs10_med_a

dfhrs10_med_a = HRS10_MEDIA_A(dfhrs10_med_m)

###-- Derivados máximos y mínimos
#Humedad relativa del aire a 10 cm  mínima diaria
df_example_c = df_example.copy()
def HRS10_MN_D(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
    # Convertir la columna de fecha a datetime si aún no lo es
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
    # Se llama la función de procesamiento de frecuencias
    df_c, complet_dia, _, _ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)

    if df_c is None or complet_dia is None:
        return None

    # Filtrar los días que tienen suficientes datos
    df_c['Fecha_temp'] = df_c.index
    dias_validos = df_c.groupby(df_c['Fecha_temp'].dt.date).filter(complet_dia).index
    
    # Filtrar el DataFrame original para incluir solo los días válidos
    dias_validos = pd.to_datetime(dias_validos).normalize()
    df_filtrado = df_c[df_c.index.normalize().isin(dias_validos)]
    
    # Encontrar el valor mínimo por cada día válido
    idx_minimos_dia = df_filtrado.groupby(df_filtrado.index.to_series().dt.date)[columna_valor].idxmin()
    mn_d = df_filtrado.loc[idx_minimos_dia]
    
    # Eliminar la columna temporal antes de retornar el resultado
    if 'Fecha_temp' in mn_d.columns:
        mn_d.drop(columns=['Fecha_temp'], inplace=True)
    
    return mn_d[[columna_valor]]

df_mn_d = HRS10_MN_D(df_example_c)

# Humedad relativa del aire a 10 cm  máxima diaria
df_example_c = df_example.copy()
def HRS10_MX_D(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
    # Convertir la columna de fecha a datetime si aún no lo es
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
    # Se llama la función de procesamiento de frecuencias
    df_c, complet_dia, _, _ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)

    if df_c is None or complet_dia is None:
        return None

    # Filtrar los días que tienen suficientes datos
    df_c['Fecha_temp'] = df_c.index
    dias_validos = df_c.groupby(df_c['Fecha_temp'].dt.date).filter(complet_dia).index
    # Filtrar el DataFrame original para incluir solo los días válidos
    dias_validos = pd.to_datetime(dias_validos).normalize()
    df_filtrado = df_c[df_c.index.normalize().isin(dias_validos)]
    # Encontrar el valor mínimo por cada día válido
    idx_maximos_dia = df_filtrado.groupby(df_filtrado.index.to_series().dt.date)[columna_valor].idxmax()
    mx_d = df_filtrado.loc[idx_maximos_dia]
    
    # Eliminar la columna temporal antes de retornar el resultado
    if 'Fecha_temp' in mx_d.columns:
        mx_d.drop(columns=['Fecha_temp'], inplace=True)
    
    return mx_d[[columna_valor]]

df_mx_d = HRS10_MX_D(df_example_c)

# Humedad relativa del aire a 10 cm  mínima mensual
df_example_c = df_example.copy()
def HRS10_MN_M(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
    # Convertir la columna de fecha a datetime si aún no lo es
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
        
    # Se llama la función de procesamiento de frecuencias
    df_c, _, complet_mes, _ = process_frequencies(df, columna_fecha, freq_csv_path,porc_min)

    if df_c is None or complet_mes is None:
        return None

    # Filtrar los meses que tienen suficientes datos
    df_c['Fecha_temp'] = df_c.index
    # Eliminar duplicados en el índice
    df_c = df_c[~df_c.index.duplicated(keep='first')]
    meses_validos = df_c.groupby(df_c.index.to_period('M')).filter(complet_mes).index.to_period('M')

    # Filtrar el DataFrame original para incluir solo los meses válidos
    df_filtrado = df_c[df_c.index.to_period('M').isin(meses_validos)]

    # Encontrar el valor mínimo por cada mes válido
    idx_minimos_mes = df_filtrado.groupby(df_filtrado.index.to_period('M'))[columna_valor].idxmin()
    min_m = df_filtrado.loc[idx_minimos_mes]

    # Eliminar las columnas temporales antes de retornar el resultado
    if 'Fecha_temp' in min_m.columns:
        min_m.drop(columns=['Fecha_temp'], inplace=True)

    return min_m[[columna_valor]]
    
df_mn_m = HRS10_MN_M(df_example_c)

# Humedad relativa del aire a 10 cm  máxima mensual
df_example_c = df_example.copy()
def HRS10_MX_M(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
    # Convertir la columna de fecha a datetime si aún no lo es
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
    
    # Se llama la función de procesamiento de frecuencias
    df_c, _, complet_mes, _ = process_frequencies(df, columna_fecha, freq_csv_path,porc_min)

    if df_c is None or complet_mes is None:
        return None

    # Filtrar los meses que tienen suficientes datos
    df_c['Fecha_temp'] = df_c.index
    # Eliminar duplicados en el índice
    df_c = df_c[~df_c.index.duplicated(keep='first')]
    meses_validos = df_c.groupby(df_c.index.to_period('M')).filter(complet_mes).index.to_period('M')

    # Filtrar el DataFrame original para incluir solo los meses válidos
    df_filtrado = df_c[df_c.index.to_period('M').isin(meses_validos)]

    # Encontrar el valor mínimo por cada mes válido
    idx_maximos_mes = df_filtrado.groupby(df_filtrado.index.to_period('M'))[columna_valor].idxmax()
    max_m = df_filtrado.loc[idx_maximos_mes]

    # Eliminar las columnas temporales antes de retornar el resultado
    if 'Fecha_temp' in max_m.columns:
        max_m.drop(columns=['Fecha_temp'], inplace=True)

    return max_m[[columna_valor]]
    
df_mx_m = HRS10_MX_M(df_example_c)

# Humedad relativa del aire a 10 cm mínima anual
df_example_c = df_example.copy()
def HRS10_MN_A(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
    # Convertir la columna de fecha a datetime si aún no lo es
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
        
    # Se llama la función de procesamiento de frecuencias
    df_c, _, _, complet_anio = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)

    if df_c is None or complet_anio is None:
        return None

    # Filtrar los meses que tienen suficientes datos
    df_c['Fecha_temp'] = df_c.index
    # Eliminar duplicados en el índice
    df_c = df_c[~df_c.index.duplicated(keep='first')]
    anios_validos = df_c.groupby(df_c.index.to_period('A')).filter(complet_anio).index.to_period('A')

    # Filtrar el DataFrame original para incluir solo los meses válidos
    df_filtrado = df_c[df_c.index.to_period('A').isin(anios_validos)]

    # Encontrar el valor mínimo por cada mes válido
    idx_minimos_anio = df_filtrado.groupby(df_filtrado.index.to_period('A'))[columna_valor].idxmin()
    min_a = df_filtrado.loc[idx_minimos_anio]

    # Eliminar las columnas temporales antes de retornar el resultado
    if 'Fecha_temp' in min_a.columns:
        min_a.drop(columns=['Fecha_temp'], inplace=True)

    return min_a[[columna_valor]]
    
df_mn_a = HRS10_MN_A(df_example_c)

# Humedad relativa del aire a 10 cm  máxima anual
df_example_c = df_example.copy()
def HRS10_MX_A(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
    # Convertir la columna de fecha a datetime si aún no lo es
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
        
    # Se llama la función de procesamiento de frecuencias
    df_c, _, _, complet_anio = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)

    if df_c is None or complet_anio is None:
        return None

    # Filtrar los meses que tienen suficientes datos
    df_c['Fecha_temp'] = df_c.index
    # Eliminar duplicados en el índice
    df_c = df_c[~df_c.index.duplicated(keep='first')]
    anios_validos = df_c.groupby(df_c.index.to_period('A')).filter(complet_anio).index.to_period('A')

    # Filtrar el DataFrame original para incluir solo los meses válidos
    df_filtrado = df_c[df_c.index.to_period('A').isin(anios_validos)]

    # Encontrar el valor mínimo por cada mes válido
    idx_minimos_anio = df_filtrado.groupby(df_filtrado.index.to_period('A'))[columna_valor].idxmax()
    max_a = df_filtrado.loc[idx_minimos_anio]

    # Eliminar las columnas temporales antes de retornar el resultado
    if 'Fecha_temp' in max_a.columns:
        max_a.drop(columns=['Fecha_temp'], inplace=True)

    return max_a[[columna_valor]]
    
df_mx_a = HRS10_MN_A(df_example_c)

In [11]:
## -- Gráficas y export en excel
# Se crea un nuevo archivo Excel con openpyxl
wb = Workbook()
sheets = {
    'HRS10_MEDIA_D': dfhrs10_med_d, 'HRS10_MEDIA_M': dfhrs10_med_m,
    'HRS10_MEDIA_A': dfhrs10_med_a, 'HRS10_MN_D': df_mn_d,
    'HRS10_MX_D': df_mx_d, 'HRS10_MN_M': df_mn_m,
    'HRS10_MX_M': df_mx_m, 'HRS10_MN_A': df_mn_a,
    'HRS10_MX_A': df_mx_a 
}

# Si el workbook todavía tiene la hoja por defecto, se elimina
if "Sheet" in wb.sheetnames:
    del wb["Sheet"]

for sheet_name, data in sheets.items():
    ws = wb.create_sheet(title=sheet_name)
    
    # Agregamos los datos al Excel
    for r_idx, row in enumerate(dataframe_to_rows(data, index=True, header=True), 1):
        for c_idx, value in enumerate(row, 1):
            ws.cell(row=r_idx, column=c_idx, value=value)
    
    # Crear una gráfica
    chart = LineChart()
    chart.title = sheet_name
    chart.style = 5
    chart.y_axis.title = 'Humedad relativa (%)'
    chart.x_axis.title = 'Fecha'
    
    # Establecer datos para la gráfica
    max_row = ws.max_row
    values = Reference(ws, min_col=2, min_row=2, max_col=2, max_row=max_row)
    dates = Reference(ws, min_col=1, min_row=3, max_col=1, max_row=max_row)
    chart.add_data(values, titles_from_data=True)
    chart.set_categories(dates)
    
    # Quitar la leyenda
    chart.legend = None
        
    # Posicionar la gráfica en el Excel
    ws.add_chart(chart, "E3")

# Guardar el archivo Excel
wb.save("Agreg_graf_HRS10_AUT_60_5T_cohr5vals.xlsx")

## Función para cálculo masivo de derivadas y generación de gráficas

In [3]:
# Función para procesar frecuencias
def process_frequencies(df, columna_fecha, freq_csv_path, porc_min=0.67):
    # Convertir la columna de fecha a datetime si aún no lo es
    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
        
    # Cargar el archivo de frecuencias
    freqinst200b = pd.read_csv(freq_csv_path, encoding='latin-1', sep=';')

    # Definir el diccionario de frecuencias y cantidades esperadas
    frecuencias = {
        'min': {'cant_esperd_h': 60, 'cant_esperd_d': 1440, 'cant_esperd_m': 43200, 'cant_esperd_a': 518400, 'minutos': 1},
        '5min': {'cant_esperd_h': 12, 'cant_esperd_d': 288, 'cant_esperd_m': 8640, 'cant_esperd_a': 103680, 'minutos': 5},
        '10min': {'cant_esperd_h': 6, 'cant_esperd_d': 144, 'cant_esperd_m': 4320, 'cant_esperd_a': 51840, 'minutos': 10},
        'h': {'cant_esperd_h': 1, 'cant_esperd_d': 24, 'cant_esperd_m': 720, 'cant_esperd_a': 8640}
    }

    # Obtener el valor de la estación
    station_value = df['station'].values[0] #Station
    freqinst200b_station = freqinst200b.loc[freqinst200b['station'] == station_value]  #Station
    periodos = freqinst200b_station['FreqInf'].values[0]

    if pd.isna(periodos):
        try:
            periodos = pd.infer_freq(df[columna_fecha][-25:])
            print(periodos)
            if periodos is None:
                project_value = freqinst100b_station['Instituc'].values[0]
                periodos = {'CENICAFE': '5min', 'IDEAM': '10min', 'CAR': 'h', 'IDIGER': 'min'}.get(project_value, 'min')
                print(f"Frecuencia inferida para {df['Station']} es None. Se determina según entidad {project_value}")
                
                #return None, None, None, None, None, None
        except ValueError as e:
            print(f'Error al inferir la frecuencia en el archivo {df}: {str(e)}')
            return None, None, None, None, None, None

    # Obtener las cantidades esperadas y el offset en minutos
    cant_esperd_h = frecuencias[periodos]['cant_esperd_h']
    cant_esperd_d = frecuencias[periodos]['cant_esperd_d']
    cant_esperd_m = frecuencias[periodos]['cant_esperd_m']
    cant_esperd_a = frecuencias[periodos]['cant_esperd_a']
    if periodos == 'h':
        pass
    else:
        minutoffset = frecuencias[periodos]['minutos']

    # Ajustar la hora de cada registro
    df_c = df.copy()
    if periodos == 'h':
        df_c[columna_fecha] = df_c[columna_fecha] #- pd.Timedelta(hours=1)
    else:
        df_c[columna_fecha] = df_c[columna_fecha] - pd.Timedelta(minutes=minutoffset)

    # Establecer la columna de fecha como índice
    df_c.set_index(columna_fecha, inplace=True)

    # Función para verificar si un día, mes o año tiene suficientes datos
    def complet_hora(sub_df):
        return len(sub_df) >= cant_esperd_h * porc_min
    
    def complet_dia(sub_df):
        return len(sub_df) >= cant_esperd_d * porc_min

    def complet_mes(sub_df):
        return len(sub_df) >= cant_esperd_m * porc_min

    def complet_anio(sub_df):
        return len(sub_df) >= cant_esperd_a * porc_min

    return df_c, periodos, complet_hora, complet_dia, complet_mes, complet_anio

# # Uso funciones de frecuencias
# df_example = pd.read_csv('../../OE_3_QC_Variables/2_HumedadRelativa/Test_QC/Estacion_0011115501.csv', encoding='latin-1')#, dtype={'Estado_Anterior':str})
# if 'Estado' in df_example.columns:
#     df_example = df_example[df_example['Estado'].apply(lambda x: any([str(x).startswith(prefix) for prefix in ['0PC']]))]    
columna_fecha = 'event_time' # Fecha
freq_csv_path = '../../../OE_3_QC_Variables/4_HumedadSuelo/HRS10/EMAHRS10_LatLonEntFreq.csv'

In [19]:
# Función para cálculo de derivadas de varios archivos en una sola carpeta
def calc_deriv_HRS10(carpeta, chunk_size=540000):
    archivos = os.listdir(carpeta)

    # Se recorre cada archivo en la carpeta
    for archivo in archivos:
        if archivo.endswith('.csv'):
            ruta_archivo = os.path.join(carpeta, archivo)
        
            # Se procesan los archivos csv por fragmentos
            reader = pd.read_csv(ruta_archivo, encoding='latin-1', chunksize=chunk_size)
            
            for chunk in reader:
                # Se generan dataframes analizados
                # De cada chunk se transforma a datetime la serie/columna 'Fecha'
                try:
                    chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%Y-%m-%d %H:%M:%S.%f')
                except ValueError:
                    chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%Y-%m-%d %H:%M:%S')

                if 'Estado' in chunk.columns:
                    try:
                        dfC = chunk[~chunk['Estado'].apply(lambda x: any([str(x).startswith(prefix) for prefix in ['0PSO0','0PAT','0PER']]))]
                        dfC_c = dfC.copy()
                        station_value = dfC_c['Station'].values[0]
                    except IndexError:
                        print(f"Error en el archivo {archivo}: dfC está vacío. Saltando al siguiente archivo.")
                        continue  # Sale del bucle de chunks y continúa con el siguiente archivo
                else:
                    chunk_c = chunk.copy()
                    station_value = chunk_c['Station'].values[0]

                # Humedad relativa media horaria
                def HRS10_MEDIA_H(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):  
                    # Convertir la columna de fecha a datetime si aún no lo es
                    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                        
                    # Se filtran los que hayan superaddo las pruebas
                    df_c, periodos, complet_hora, _, _, _ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)

                    # Luego de establecer el índice, aplicar resample
                    df_filtrado = df_c.groupby([df_c.index.date, df_c.index.hour]).filter(complet_hora)
                    hrs10_med_h = df_filtrado[['Valor']].resample('h').mean()
                
                    return hrs10_med_h[['Valor']]

                # Se llama el archivo original según si es dato crudo o con qc
                if 'Estado' in chunk.columns:
                    dfhrs10_med_h = HRS10_MEDIA_H(dfC_c)
                else:
                    dfhrs10_med_h = HRS10_MEDIA_H(chunk_c)
                
                ## Derivadas diarias, mensuales y anuales
                # Humedad relativa del aire a 10 cm media diaria
                def HRS10_MEDIA_D(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
                    df.reset_index(inplace=True)
                    # Convertir la columna de fecha a datetime si aún no lo es
                    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                    
                    # Cantidad esperada por día
                    cant_esperd_d = 24
                    # Función para verificar si una hora específica tiene suficientes datos
                    def complet_dia(sub_df, total_esperado=cant_esperd_d, porc_min=porc_min):
                        return len(sub_df) >= total_esperado * porc_min
                    
                    df.set_index('Fecha', inplace=True)
                    # Se filtran los datos que tienen la complititud mínima
                    df_filtrado = df.groupby([df.index.date]).filter(complet_dia) #dfC.groupby([dfC.index.date]).filter(complet_dia)
                    hrs10_med_d = df_filtrado[[columna_valor]].resample('D').mean()
                
                    return hrs10_med_d
                
                dfhrs10_med_d = HRS10_MEDIA_D(dfhrs10_med_h)                
                # Se llama el archivo original según si es dato crudo o con qc
                #if 'Estado' in chunk.columns:
                #    dfhrs10_med_d = HRS10_MEDIA_D(dfC_c)
                #else:
                #    dfhrs10_med_d = HRS10_MEDIA_D(chunk_c)
                
                # Presión atmosférica media mensual
                def HRS10_MEDIA_M(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):    
                    if df is None:
                        return None                  
                    days_in_month = df.index.to_series().dt.days_in_month
                    days_in_month = days_in_month.resample('ME').first()
                    
                    # Función para verificar si una hora específica tiene suficientes datos
                    def complet_mes(sub_df):
                        mes = sub_df.index[0].month
                        total_esperado = days_in_month[days_in_month.index.month == mes].iloc[0]
                        return len(sub_df) >= total_esperado * porc_min
                
                    # Luego de establecer el índice, aplicar resample
                    df_filtrado = df.groupby([df.index.year, df.index.month]).filter(complet_mes)
                    hrs10_med_m = df_filtrado[['Valor']].resample('ME').mean()
                    
                    return hrs10_med_m
                
                dfhrs10_med_m = HRS10_MEDIA_M(dfhrs10_med_d)
                
                # Presión atmosférica del Aire a 2 metros media anual
                def HRS10_MEDIA_A(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
                    if df is None:
                        return None
                    
                    # Función para verificar si una hora específica tiene suficientes datos
                    def complet_anio(sub_df):
                        return len(sub_df) >= 12 * porc_min
                
                    # Luego de establecer el índice, aplicar resample
                    df_filtrado = df.groupby([df.index.year]).filter(complet_anio)
                    hrs10_med_a = df_filtrado[['Valor']].resample('YE').mean()
                    
                    return hrs10_med_a
                
                dfhrs10_med_a = HRS10_MEDIA_A(dfhrs10_med_m)
                
                ###-- Derivados máximos y mínimos
                #Humedad relativa del aire a 10 cm  mínima diaria
                def HRS10_MN_H(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
                    # Convertir la columna de fecha a datetime si aún no lo es
                    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                    # Se llama la función de procesamiento de frecuencias
                    df_c, periodo, complet_hora, _, _, _ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)
                
                    if df_c is None or complet_hora is None:
                        return None
                
                    # Filtrar los días que tienen suficientes datos
                    df_c['Fecha_temp'] = df_c.index
                    horas_validas = df_c.groupby(df_c['Fecha_temp'].dt.date).filter(complet_hora).index
                    
                    # Filtrar el DataFrame original para incluir solo los días válidos
                    horas_validas = pd.to_datetime(horas_validas).normalize()
                    df_filtrado = df_c[df_c.index.normalize().isin(horas_validas)]
                    
                    # Encontrar el valor mínimo por cada día válido
                    idx_minimos_hora = df_filtrado.groupby(df_filtrado.index.to_series().dt.date)[columna_valor].idxmin()
                    mn_h = df_filtrado.loc[idx_minimos_hora]
                    
                    # Eliminar la columna temporal antes de retornar el resultado
                    if 'Fecha_temp' in mn_h.columns:
                        mn_h.drop(columns=['Fecha_temp'], inplace=True)
                    
                    return mn_h[[columna_valor]]

                # Se llama el archivo original según si es dato crudo o con qc
                if 'Estado' in chunk.columns:
                    df_mn_h = HRS10_MN_H(dfC_c)
                else:
                    df_mn_h = HRS10_MN_H(chunk_c)
                    
                #Humedad relativa del aire a 10 cm  mínima diaria
                def HRS10_MX_H(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
                    # Convertir la columna de fecha a datetime si aún no lo es
                    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                    # Se llama la función de procesamiento de frecuencias
                    df_c, periodo, complet_hora, _, _, _ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)
                
                    if df_c is None or complet_hora is None:
                        return None
                
                    # Filtrar los días que tienen suficientes datos
                    df_c['Fecha_temp'] = df_c.index
                    horas_validas = df_c.groupby(df_c['Fecha_temp'].dt.date).filter(complet_hora).index
                    
                    # Filtrar el DataFrame original para incluir solo los días válidos
                    horas_validas = pd.to_datetime(horas_validas).normalize()
                    df_filtrado = df_c[df_c.index.normalize().isin(horas_validas)]
                    
                    # Encontrar el valor mínimo por cada día válido
                    idx_maximos_hora = df_filtrado.groupby(df_filtrado.index.to_series().dt.date)[columna_valor].idxmax()
                    mx_h = df_filtrado.loc[idx_maximos_hora]
                    
                    # Eliminar la columna temporal antes de retornar el resultado
                    if 'Fecha_temp' in mx_h.columns:
                        mx_h.drop(columns=['Fecha_temp'], inplace=True)
                    
                    return mx_h[[columna_valor]]

                # Se llama el archivo original según si es dato crudo o con qc
                if 'Estado' in chunk.columns:
                    df_mx_h = HRS10_MX_H(dfC_c)
                else:
                    df_mx_h = HRS10_MX_H(chunk_c)
                
                def HRS10_MN_D(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
                    # Convertir la columna de fecha a datetime si aún no lo es
                    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                    # Se llama la función de procesamiento de frecuencias
                    df_c, periodo,_,complet_dia, _, _ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)
                
                    if df_c is None or complet_dia is None:
                        return None
                
                    # Filtrar los días que tienen suficientes datos
                    df_c['Fecha_temp'] = df_c.index
                    dias_validos = df_c.groupby(df_c['Fecha_temp'].dt.date).filter(complet_dia).index
                    
                    # Filtrar el DataFrame original para incluir solo los días válidos
                    dias_validos = pd.to_datetime(dias_validos).normalize()
                    df_filtrado = df_c[df_c.index.normalize().isin(dias_validos)]
                    
                    # Encontrar el valor mínimo por cada día válido
                    idx_minimos_dia = df_filtrado.groupby(df_filtrado.index.to_series().dt.date)[columna_valor].idxmin()
                    mn_d = df_filtrado.loc[idx_minimos_dia]
                    
                    # Eliminar la columna temporal antes de retornar el resultado
                    if 'Fecha_temp' in mn_d.columns:
                        mn_d.drop(columns=['Fecha_temp'], inplace=True)
                    
                    return mn_d[[columna_valor]]

                # Se llama el archivo original según si es dato crudo o con qc
                if 'Estado' in chunk.columns:
                    df_mn_d = HRS10_MN_D(dfC_c)
                else:
                    df_mn_d = HRS10_MN_D(chunk_c)
                
                # Humedad relativa del aire a 10 cm  máxima diaria
                def HRS10_MX_D(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
                    # Convertir la columna de fecha a datetime si aún no lo es
                    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                    # Se llama la función de procesamiento de frecuencias
                    df_c, periodo,_,complet_dia,_,_ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)
                
                    if df_c is None or complet_dia is None:
                        return None
                
                    # Filtrar los días que tienen suficientes datos
                    df_c['Fecha_temp'] = df_c.index
                    dias_validos = df_c.groupby(df_c['Fecha_temp'].dt.date).filter(complet_dia).index
                    # Filtrar el DataFrame original para incluir solo los días válidos
                    dias_validos = pd.to_datetime(dias_validos).normalize()
                    df_filtrado = df_c[df_c.index.normalize().isin(dias_validos)]
                    # Encontrar el valor mínimo por cada día válido
                    idx_maximos_dia = df_filtrado.groupby(df_filtrado.index.to_series().dt.date)[columna_valor].idxmax()
                    mx_d = df_filtrado.loc[idx_maximos_dia]
                    
                    # Eliminar la columna temporal antes de retornar el resultado
                    if 'Fecha_temp' in mx_d.columns:
                        mx_d.drop(columns=['Fecha_temp'], inplace=True)
                    
                    return mx_d[[columna_valor]]
                
                # Se llama el archivo original según si es dato crudo o con qc
                if 'Estado' in chunk.columns:
                    df_mx_d = HRS10_MX_D(dfC_c)
                else:
                    df_mx_d = HRS10_MX_D(chunk_c)
                
                # Humedad relativa del aire a 10 cm  mínima mensual
                def HRS10_MN_M(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
                    # Convertir la columna de fecha a datetime si aún no lo es
                    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                    # Se llama la función de procesamiento de frecuencias
                    df_c, periodo,_,_,complet_mes,_ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)
                
                    if df_c is None or complet_mes is None:
                        return None
                
                    # Filtrar los meses que tienen suficientes datos
                    df_c['Fecha_temp'] = df_c.index
                    # Eliminar duplicados en el índice
                    df_c = df_c[~df_c.index.duplicated(keep='first')]
                    meses_validos = df_c.groupby(df_c.index.to_period('M')).filter(complet_mes).index.to_period('M')
                
                    # Filtrar el DataFrame original para incluir solo los meses válidos
                    df_filtrado = df_c[df_c.index.to_period('M').isin(meses_validos)]
                
                    # Encontrar el valor mínimo por cada mes válido
                    idx_minimos_mes = df_filtrado.groupby(df_filtrado.index.to_period('M'))[columna_valor].idxmin()
                    min_m = df_filtrado.loc[idx_minimos_mes]
                
                    # Eliminar las columnas temporales antes de retornar el resultado
                    if 'Fecha_temp' in min_m.columns:
                        min_m.drop(columns=['Fecha_temp'], inplace=True)
                
                    return min_m[[columna_valor]]

                # Se llama el archivo original según si es dato crudo o con qc
                if 'Estado' in chunk.columns:
                    df_mn_m = HRS10_MN_M(dfC_c)
                else:
                    df_mn_m = HRS10_MN_M(chunk_c)
                
                # Humedad relativa del aire a 10 cm  máxima mensual
                def HRS10_MX_M(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
                    # Convertir la columna de fecha a datetime si aún no lo es
                    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                    # Se llama la función de procesamiento de frecuencias
                    df_c, periodo,_,_,complet_mes,_ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)
                
                    if df_c is None or complet_mes is None:
                        return None
                
                    # Filtrar los meses que tienen suficientes datos
                    df_c['Fecha_temp'] = df_c.index
                    # Eliminar duplicados en el índice
                    df_c = df_c[~df_c.index.duplicated(keep='first')]
                    meses_validos = df_c.groupby(df_c.index.to_period('M')).filter(complet_mes).index.to_period('M')
                
                    # Filtrar el DataFrame original para incluir solo los meses válidos
                    df_filtrado = df_c[df_c.index.to_period('M').isin(meses_validos)]
                
                    # Encontrar el valor mínimo por cada mes válido
                    idx_maximos_mes = df_filtrado.groupby(df_filtrado.index.to_period('M'))[columna_valor].idxmax()
                    max_m = df_filtrado.loc[idx_maximos_mes]
                
                    # Eliminar las columnas temporales antes de retornar el resultado
                    if 'Fecha_temp' in max_m.columns:
                        max_m.drop(columns=['Fecha_temp'], inplace=True)
                
                    return max_m[[columna_valor]]

                # Se llama el archivo original según si es dato crudo o con qc
                if 'Estado' in chunk.columns:
                    df_mx_m = HRS10_MX_M(dfC_c)
                else:
                    df_mx_m = HRS10_MX_M(chunk_c)
                
                # Humedad relativa del aire a 10 cm mínima anual
                def HRS10_MN_A(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
                    # Convertir la columna de fecha a datetime si aún no lo es
                    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                    # Se llama la función de procesamiento de frecuencias
                    df_c, periodo,_,_,_,complet_anio = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)
                
                    if df_c is None or complet_anio is None:
                        return None
                
                    # Filtrar los meses que tienen suficientes datos
                    df_c['Fecha_temp'] = df_c.index
                    # Eliminar duplicados en el índice
                    df_c = df_c[~df_c.index.duplicated(keep='first')]
                    anios_validos = df_c.groupby(df_c.index.to_period('Y')).filter(complet_anio).index.to_period('Y')
                
                    # Filtrar el DataFrame original para incluir solo los meses válidos
                    df_filtrado = df_c[df_c.index.to_period('Y').isin(anios_validos)]
                
                    # Encontrar el valor mínimo por cada mes válido
                    idx_minimos_anio = df_filtrado.groupby(df_filtrado.index.to_period('Y'))[columna_valor].idxmin()
                    min_a = df_filtrado.loc[idx_minimos_anio]
                
                    # Eliminar las columnas temporales antes de retornar el resultado
                    if 'Fecha_temp' in min_a.columns:
                        min_a.drop(columns=['Fecha_temp'], inplace=True)
                
                    return min_a[[columna_valor]]

                # Se llama el archivo original según si es dato crudo o con qc
                if 'Estado' in chunk.columns:
                    df_mn_a = HRS10_MN_A(dfC_c)
                else:
                    df_mn_a = HRS10_MN_A(chunk_c)
                
                # Humedad relativa del aire a 10 cm  máxima anual
                def HRS10_MX_A(df, columna_fecha='Fecha', columna_valor='Valor', porc_min=0.67):
                    # Convertir la columna de fecha a datetime si aún no lo es
                    if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                        df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                    # Se llama la función de procesamiento de frecuencias
                    df_c, periodo,_,_,_,complet_anio = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)
                
                    if df_c is None or complet_anio is None:
                        return None
                
                    # Filtrar los meses que tienen suficientes datos
                    df_c['Fecha_temp'] = df_c.index
                    # Eliminar duplicados en el índice
                    df_c = df_c[~df_c.index.duplicated(keep='first')]
                    anios_validos = df_c.groupby(df_c.index.to_period('Y')).filter(complet_anio).index.to_period('Y')
                
                    # Filtrar el DataFrame original para incluir solo los meses válidos
                    df_filtrado = df_c[df_c.index.to_period('Y').isin(anios_validos)]
                
                    # Encontrar el valor mínimo por cada mes válido
                    idx_maximos_anio = df_filtrado.groupby(df_filtrado.index.to_period('Y'))[columna_valor].idxmax()
                    max_a = df_filtrado.loc[idx_maximos_anio]
                
                    # Eliminar las columnas temporales antes de retornar el resultado
                    if 'Fecha_temp' in max_a.columns:
                        max_a.drop(columns=['Fecha_temp'], inplace=True)
                
                    return max_a[[columna_valor]]
                    
                # Se llama el archivo original según si es dato crudo o con qc
                if 'Estado' in chunk.columns:
                    df_mx_a = HRS10_MX_A(dfC_c)
                else:
                    df_mx_a = HRS10_MX_A(chunk_c)
        
                ### Se crea un nuevo archivo Excel con openpyxl
                wb = Workbook()
                sheets = {
                    'HRS10_MEDIA_H': dfhrs10_med_h, 'HRS10_MEDIA_D': dfhrs10_med_d,
                    'HRS10_MEDIA_M': dfhrs10_med_m, 'HRS10_MEDIA_A': dfhrs10_med_a, 
                    'HRS10_MN_H': df_mn_h,'HRS10_MX_H': df_mx_h,
                    'HRS10_MN_D': df_mn_d,'HRS10_MX_D': df_mx_d, 
                    'HRS10_MN_M': df_mn_m,'HRS10_MX_M': df_mx_m, 
                    'HRS10_MN_A': df_mn_a,'HRS10_MX_A': df_mx_a 
                }
                
                # Si el workbook todavía tiene la hoja por defecto, se elimina
                if "Sheet" in wb.sheetnames:
                    del wb["Sheet"]
                
                for sheet_name, data in sheets.items():
                    ws = wb.create_sheet(title=sheet_name)
                    
                    # Agregamos los datos al Excel
                    if data is None:
                        pass
                    else:
                        for r_idx, row in enumerate(dataframe_to_rows(data, index=True, header=True), 1):
                            for c_idx, value in enumerate(row, 1):
                                ws.cell(row=r_idx, column=c_idx, value=value)
                    
                    # Crear una gráfica
                    chart = LineChart()
                    chart.title = sheet_name
                    chart.style = 5
                    chart.y_axis.title = 'Humedad del suelo -10cm (%)'
                    chart.x_axis.title = 'Fecha'
                    
                    # Establecer datos para la gráfica
                    max_row = ws.max_row
                    values = Reference(ws, min_col=2, min_row=2, max_col=2, max_row=max_row)
                    dates = Reference(ws, min_col=1, min_row=3, max_col=1, max_row=max_row)
                    chart.add_data(values, titles_from_data=True)
                    chart.set_categories(dates)
                    
                    # Quitar la leyenda
                    chart.legend = None
                    
                    # Cambiar el grosor de la línea a 0.5 puntos (equivalente a 50 centésimas de punto)
                    for series in chart.series:
                        series.graphicalProperties.line.width = 50
                        series.graphicalProperties.line.solidFill = "5499c7"
                        
                    # Posicionar la gráfica en el Excel
                    ws.add_chart(chart, "E3")
                
                # Nombres archivos
                if 'Estado' in chunk.columns:
                    nombre_archivo_salida = os.path.join(carpeta, archivo[:22] + '_deriv.xlsx')
                else:
                    nombre_archivo_salida = os.path.join(carpeta, archivo[:19] + '_deriv.xlsx')
                #nombre_archivo_salida = os.path.join(carpeta, archivo[:22] + '_deriv.xlsx') #archivo[:19] el original #archivo[:22] con qc
                
                # Guardar el archivo Excel
                wb.save(nombre_archivo_salida)

In [21]:
calc_deriv_HRS10('../../../OE_3_QC_Variables/4_HumedadSuelo/HRS10/QCResult_HRS10')

C:\Users\palvarez\AppData\Local\Temp\ipykernel_4440\3456756448.py:13: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
C:\Users\palvarez\AppData\Local\Temp\ipykernel_4440\3456756448.py:13: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
C:\Users\palvarez\AppData\Local\Temp\ipykernel_4440\3456756448.py:13: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:


Error en el archivo Estacion_0023195040_qc.csv: dfC está vacío. Saltando al siguiente archivo.
Error en el archivo Estacion_0024015110_qc.csv: dfC está vacío. Saltando al siguiente archivo.
Error en el archivo Estacion_0026015010_qc.csv: dfC está vacío. Saltando al siguiente archivo.
Error en el archivo Estacion_0026015040_qc.csv: dfC está vacío. Saltando al siguiente archivo.
Error en el archivo Estacion_0026055120_qc.csv: dfC está vacío. Saltando al siguiente archivo.


C:\Users\palvarez\AppData\Local\Temp\ipykernel_4440\3456756448.py:13: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
C:\Users\palvarez\AppData\Local\Temp\ipykernel_4440\3456756448.py:13: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
C:\Users\palvarez\AppData\Local\Temp\ipykernel_4440\3456756448.py:13: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:


Error en el archivo Estacion_0035017020_qc.csv: dfC está vacío. Saltando al siguiente archivo.
Error en el archivo Estacion_0046015030_qc.csv: dfC está vacío. Saltando al siguiente archivo.


C:\Users\palvarez\AppData\Local\Temp\ipykernel_4440\3456756448.py:13: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:


Error en el archivo Estacion_0054017040_qc.csv: dfC está vacío. Saltando al siguiente archivo.


In [22]:
calc_deriv_HRS10('../../../OE_3_QC_Variables/4_HumedadSuelo/HRS10/RawUnmodified_HRS10')

----

## Función cálculo masivo de derivadas y alistamiento Cassandra

In [5]:
def calc_deriv_HRS10_QC(carpeta):#, chunk_size=540000):
    archivos = os.listdir(carpeta)

    # Se crean carpetas para cada tipo de DataFrame si no existen
    carpetas_salidas = ['HRS10_MEDIA_H_QC', 'HRS10_MEDIA_D_QC', 'HRS10_MEDIA_M_QC', 'HRS10_MEDIA_A_QC']#,
                        #'HRS10_MN_H_QC', 'HRS10_MX_H_QC', 'HRS10_MN_D_QC', 'HRS10_MX_D_QC',
                        #'HRS10_MN_M_QC', 'HRS10_MX_M_QC', 'HRS10_MN_A_QC', 'HRS10_MX_A_QC']

    for cs in carpetas_salidas:
        os.makedirs(os.path.join(carpeta, cs), exist_ok=True)
    
    # Procesar cada archivo en la carpeta
    for archivo in archivos:
        if archivo.endswith('.csv'):
            ruta_archivo = os.path.join(carpeta, archivo)
            df = pd.read_csv(ruta_archivo, encoding='latin-1')

            # Procesar la fecha y filtrar según estado, como en el ejemplo original
            try:
                df['event_time'] = pd.to_datetime(df['event_time'], format='%Y-%m-%d %H:%M:%S.%f')
            except ValueError:
                df['event_time'] = pd.to_datetime(df['event_time'], format='%Y-%m-%d %H:%M:%S')
                  
            try:
                dfC = df[df['state'].apply(lambda x: any([str(x).startswith(prefix) for prefix in ['0PC']]))] # Se comenta si es de crudos
                station_value = dfC['station'].values[0] #df['station'].values[0]
            except IndexError:
                print(f"Error en el archivo {archivo}: dfC está vacío. Saltando al siguiente archivo.")
                continue  # Sale del bucle de chunks y continúa con el siguiente archivo

            # Humedad relativa media horaria
            def HRS10_MEDIA_H_QC(df, columna_fecha='event_time', columna_valor='event_value', porc_min=0.67):  
                # Convertir la columna de fecha a datetime si aún no lo es
                if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                    df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                    
                # Se filtran los que hayan superaddo las pruebas
                df_c, periodos, complet_hora, _, _, _ = process_frequencies(df, columna_fecha, freq_csv_path, porc_min)

                # Luego de establecer el índice, aplicar resample
                df_filtrado = df_c.groupby([df_c.index.date, df_c.index.hour]).filter(complet_hora)
                HRS10_60_med_h = df_filtrado[['event_value']].resample('h').mean()

                # Cambio de contenido de columnas
                HRS10_60_med_h['station'] = df['station'].iloc[0]
                HRS10_60_med_h['station'] = HRS10_60_med_h['station'].astype('int64')
                HRS10_60_med_h['label'] = 'HRS10_MEDIA_H_QC'
                
                # Reset index
                HRS10_60_med_h.reset_index(inplace=True)
                
                # Se reordenan las columnas
                nuevo_orden = ['station', 'label', 'event_time', 'event_value']
                # Reordenar las columnas usando el nuevo orden
                HRS10_60_med_h = HRS10_60_med_h[nuevo_orden]
            
                # Convertir 'event_time' de nuevo a datetime para uniformidad
                HRS10_60_med_h['event_time'] = pd.to_datetime(HRS10_60_med_h['event_time'])
            
                return HRS10_60_med_h

            dfhrs10_med_h = HRS10_MEDIA_H_QC(dfC)
            
            # Humedad relativa media diaria
            def HRS10_MEDIA_D_QC(df, columna_fecha='event_time', columna_valor='event_value', porc_min=0.67):
                if df.empty:
                    return df
                df.reset_index(inplace=True)
                # Convertir la columna de fecha a datetime si aún no lo es
                if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                    df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                # Ajustar la hora de cada registro para que corresponda al rango deseado
                df[columna_fecha] = df[columna_fecha] - pd.Timedelta(hours=1)
                # Establecer la columna de fecha como índice
                df.set_index(columna_fecha, inplace=True)
                
                # Calcular el total de registros esperados por día pluviométrico
                total_esperado_por_dia = 24
                # Función para verificar si un día pluviométrico tiene suficientes datos
                def complet_dia(sub_df):
                    return len(sub_df) >= total_esperado_por_dia * porc_min
            
                # Filtrar los días con suficientes datos y calcular el promedio diario
                df_filtrado = df.groupby([df.index.date]).filter(complet_dia)
                # Verificar si el DataFrame filtrado está vacío
                if df_filtrado.empty:
                    print(f"DataFrame vacío después de filtrar por días válidos. Regresando {archivo[:19]} vacío.")
                    return df_filtrado
                    
                # Se calcula la media
                HRS10_60_med_d = df_filtrado[[columna_valor]].resample('D').mean()

                # Cambio de contenido de columnas
                HRS10_60_med_d['station'] = df['station'].iloc[0]
                HRS10_60_med_d['station'] = HRS10_60_med_d['station'].astype('int64')
                HRS10_60_med_d['label'] = 'HRS10_MEDIA_D_QC'
                
                # Reset index
                HRS10_60_med_d.reset_index(inplace=True)
                
                # Se reordenan las columnas
                nuevo_orden = ['station', 'label', 'event_time', 'event_value']
                # Reordenar las columnas usando el nuevo orden
                HRS10_60_med_d = HRS10_60_med_d[nuevo_orden]
            
                # Convertir 'event_time' de nuevo a datetime para uniformidad
                HRS10_60_med_d['event_time'] = pd.to_datetime(HRS10_60_med_d['event_time'])
            
                return HRS10_60_med_d

            dfhrs10_med_d = HRS10_MEDIA_D_QC(dfhrs10_med_h)
            
            # Humedad relativa media mensual
            dfhrs10_med_d_c = dfhrs10_med_d.copy()
            def HRS10_MEDIA_M_QC(df, columna_fecha='event_time', columna_valor='event_value', porc_min=0.67):    
                if df.empty:
                    return df
                df.reset_index(inplace=True)
                # Convertir la columna de fecha a datetime si aún no lo es
                if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                    df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                
                # Establecer la columna 'event_time' como índice
                df.set_index(columna_fecha, inplace=True)
                
                days_in_month = df.index.to_series().dt.days_in_month
                days_in_month = days_in_month.resample('ME').first()
                
                # Función para verificar si una hora específica tiene suficientes datos
                def complet_mes(sub_df):
                    mes = sub_df.index[0].month
                    total_esperado = days_in_month[days_in_month.index.month == mes].iloc[0]
                    return len(sub_df) >= total_esperado * porc_min
            
                # Luego de establecer el índice, aplicar resample
                df_filtrado = df.groupby([df.index.year, df.index.month]).filter(complet_mes)
                # Verificar si el DataFrame filtrado está vacío
                if df_filtrado.empty:
                    print(f"DataFrame vacío después de filtrar por meses válidos. Regresando {archivo[:19]} vacío.")
                    return df_filtrado
                    
                # Se calcula la media 
                HRS10_60_med_m = df_filtrado[['event_value']].resample('ME').mean()

                # Cambio de contenido de columnas
                HRS10_60_med_m['station'] = df['station'].iloc[0]
                HRS10_60_med_m['station'] = HRS10_60_med_m['station'].astype('int64')
                HRS10_60_med_m['label'] = 'HRS10_MEDIA_M_QC'
                
                # Reset index
                HRS10_60_med_m.reset_index(inplace=True)
                
                # Se reordenan las columnas
                nuevo_orden = ['station', 'label', 'event_time', 'event_value']
                # Reordenar las columnas usando el nuevo orden
                HRS10_60_med_m = HRS10_60_med_m[nuevo_orden]
            
                # Convertir 'event_time' de nuevo a datetime para uniformidad
                HRS10_60_med_m['event_time'] = pd.to_datetime(HRS10_60_med_m['event_time'])
                
                return HRS10_60_med_m
            
            dfhrs10_med_m = HRS10_MEDIA_M_QC(dfhrs10_med_d_c)
            
            # Humedad relativa media anual
            dfhrs10_med_m_c = dfhrs10_med_m.copy()
            def HRS10_MEDIA_A_QC(df, columna_fecha='event_time', columna_valor='event_value', porc_min=0.67):
                if df.empty:
                    return df
                df.reset_index(inplace=True)
                # Convertir la columna de fecha a datetime si aún no lo es
                if not pd.api.types.is_datetime64_any_dtype(df[columna_fecha]):
                    df[columna_fecha] = pd.to_datetime(df[columna_fecha])
                
                # Función para verificar si una hora específica tiene suficientes datos
                def complet_anio(sub_df):
                    return len(sub_df) >= 12 * porc_min
                   
                # Antes de resample, establecer la columna 'event_time' como índice
                df.set_index(columna_fecha, inplace=True)
            
                # Luego de establecer el índice, aplicar resample
                df_filtrado = df.groupby([df.index.year]).filter(complet_anio)
                # Verificar si el DataFrame filtrado está vacío
                if df_filtrado.empty:
                    print(f"DataFrame vacío después de filtrar por años válidos. Regresando {archivo[:19]} vacío.")
                    return df_filtrado
                    
                # Se calcula la media    
                HRS10_60_med_a = df_filtrado[['event_value']].resample('YE').mean()

                # Cambio de contenido de columnas
                HRS10_60_med_a['station'] = df['station'].iloc[0]
                HRS10_60_med_a['station'] = HRS10_60_med_a['station'].astype('int64')
                HRS10_60_med_a['label'] = 'HRS10_MEDIA_A_QC'
                
                # Reset index
                HRS10_60_med_a.reset_index(inplace=True)
                
                # Se reordenan las columnas
                nuevo_orden = ['station', 'label', 'event_time', 'event_value']
                # Reordenar las columnas usando el nuevo orden
                HRS10_60_med_a = HRS10_60_med_a[nuevo_orden]
            
                # Convertir 'event_time' de nuevo a datetime para uniformidad
                HRS10_60_med_a['event_time'] = pd.to_datetime(HRS10_60_med_a['event_time'])
                
                return HRS10_60_med_a
            
            dfhrs10_med_a = HRS10_MEDIA_A_QC(dfhrs10_med_m_c)

            if not dfhrs10_med_h.empty:
                dfhrs10_med_h.to_csv(os.path.join(carpeta, 'HRS10_MEDIA_H_QC', f'{archivo[:19]}.csv'), index=False)
            if not dfhrs10_med_d.empty:
                dfhrs10_med_d.to_csv(os.path.join(carpeta, 'HRS10_MEDIA_D_QC', f'{archivo[:19]}.csv'), index=False)
            if not dfhrs10_med_m.empty:
                dfhrs10_med_m.to_csv(os.path.join(carpeta, 'HRS10_MEDIA_M_QC', f'{archivo[:19]}.csv'), date_format='%Y-%m', index=False)
            if not dfhrs10_med_a.empty:
                dfhrs10_med_a.to_csv(os.path.join(carpeta, 'HRS10_MEDIA_A_QC', f'{archivo[:19]}.csv'), date_format='%Y',index=False)

In [7]:
calc_deriv_HRS10_QC('../../../OE_3_QC_Variables/4_HumedadSuelo/HRS10/ReadyToCassandraFiles_HRS10/')

DataFrame vacío después de filtrar por años válidos. Regresando Estacion_0011115501 vacío.
DataFrame vacío después de filtrar por años válidos. Regresando Estacion_0016025502 vacío.
DataFrame vacío después de filtrar por años válidos. Regresando Estacion_0021065501 vacío.
DataFrame vacío después de filtrar por meses válidos. Regresando Estacion_0021185090 vacío.
DataFrame vacío después de filtrar por años válidos. Regresando Estacion_0021255501 vacío.
Error en el archivo Estacion_0023195040_qc.csv: dfC está vacío. Saltando al siguiente archivo.
DataFrame vacío después de filtrar por años válidos. Regresando Estacion_0023195501 vacío.
Error en el archivo Estacion_0024015110_qc.csv: dfC está vacío. Saltando al siguiente archivo.
Error en el archivo Estacion_0026015010_qc.csv: dfC está vacío. Saltando al siguiente archivo.
Error en el archivo Estacion_0026015040_qc.csv: dfC está vacío. Saltando al siguiente archivo.
DataFrame vacío después de filtrar por días válidos. Regresando Estacion_

### Cálculo promedios horarios mensuales multianuales

In [20]:
def calc_hmMa_HRS10(carpeta, chunk_size=540000):
    archivos = os.listdir(carpeta)

    # Se recorre cada archivo en la carpeta
    for archivo in archivos:
        if archivo.endswith('.csv'):
            ruta_archivo = os.path.join(carpeta, archivo)
        
            # Se procesan los archivos csv por fragmentos
            reader = pd.read_csv(ruta_archivo, encoding='latin-1', chunksize=chunk_size)
            
            for chunk in reader:
                # Se generan dataframes analizados
                # De cada chunk se transforma a datetime la serie/columna 'Fecha'
                try:
                    chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%Y-%m-%d %H:%M:%S.%f')
                except ValueError:
                    chunk['Fecha'] = pd.to_datetime(chunk['Fecha'], format='%Y-%m-%d %H:%M:%S')
                    chunk = chunk[~chunk['Estado'].apply(lambda x: any([str(x).startswith(prefix) for prefix in ['0PSO0','0PAT','0PER']]))]

                # Se hace la agrupación para cálculo de medias horarias mensuales multianuales
                hym_ma = chunk['Valor'].groupby(by =[chunk["Fecha"].dt.month, chunk["Fecha"].dt.hour]).mean().unstack(level=0)

                # Crear un archivo Excel y agregar los datos
                wb = Workbook()
                ws = wb.active
                ws.title = "Datos"
                
                # Agregar datos al archivo Excel
                for r in dataframe_to_rows(hym_ma.reset_index(), index=False, header=True):
                    ws.append(r)
                
                # Crear la gráfica de dispersión
                chart = ScatterChart()
                chart.title = "Valores Promedio por Hora y Mes"
                chart.style = 13
                chart.x_axis.title = 'Hora del día'
                chart.y_axis.title = 'Valor'
                
                # Aumentar el tamaño del gráfico
                chart.width = 20  # Anchura del gráfico (pulgadas)
                chart.height = 12  # Altura del gráfico (pulgadas)
                
                # Fijar el máximo valor del eje x
                chart.x_axis.scaling.max = 23
                chart.x_axis.scaling.min = 0
                chart.x_axis.majorUnit = 1
                
                # Agregar series a la gráfica
                colors = ['1F77B4', 'FF7F0E', '2CA02C', 'D62728', '9467BD', '8C564B', 'E377C2', '7F7F7F', 'BCBD22', '17BECF', 'AEC7E8', 'FFBB78']
                for i in range(2, 14):  # Columnas B a M (meses 1 a 12)
                    xvalues = Reference(ws, min_col=1, min_row=2, max_row=25)
                    yvalues = Reference(ws, min_col=i, min_row=1, max_row=25)
                    series = Series(yvalues, xvalues, title_from_data=True)
                    series.graphicalProperties.line.solidFill = colors[i % len(colors)]  # Asignar colores a las líneas
                    series.graphicalProperties.line.width = 30000  # Ajustar el grosor de las líneas
                    series.marker.symbol = 'circle'  # Cambiar el marcador a círculo
                    series.marker.size = 5
                    series.marker.graphicalProperties.solidFill = colors[i % len(colors)]  # Cambiar el color del marcador
                    chart.series.append(series)
                
                # Insertar la gráfica en la hoja de cálculo
                ws.add_chart(chart, "O2")

                # Nombres archivos
                nombre_archivo_salida = os.path.join(carpeta, archivo[:19] + '_hm_ma.xlsx') #archivo[:22] el de qc
                # Guardar el archivo Excel
                wb.save(nombre_archivo_salida)

In [22]:
calc_hmMa_HRS10('../../../OE_3_QC_Variables/4_HumedadSuelo/HRS10/QCResult_HRS10/')  #RawUnmodified_HRS10

C:\Users\palvarez\AppData\Local\Temp\ipykernel_16064\2946228248.py:12: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
C:\Users\palvarez\AppData\Local\Temp\ipykernel_16064\2946228248.py:12: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
C:\Users\palvarez\AppData\Local\Temp\ipykernel_16064\2946228248.py:12: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
C:\Users\palvarez\AppData\Local\Temp\ipykernel_16064\2946228248.py:12: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
C:\Users\palvarez\AppData\Local\Temp\ipykernel_16064\2946228248.py:12: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in reader:
C:\Users\palvarez\AppData\Loca

In [23]:
calc_hmMa_HRS10('../../../OE_3_QC_Variables/4_HumedadSuelo/HRS10/RawUnmodified_HRS10/')